# Downloads Qtacotron (need fix)

In [1]:
!gdown --id 1Lo4ECKs5n8xoOD2SaGV-kup1yZcZMTwk

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Lo4ECKs5n8xoOD2SaGV-kup1yZcZMTwk
To: /content/qtacotron.zip
100% 104M/104M [00:01<00:00, 56.6MB/s] 


In [2]:
!unzip /content/qtacotron.zip

Archive:  /content/qtacotron.zip
   creating: qtacotron/
  inflating: __MACOSX/._qtacotron    
  inflating: qtacotron/vocoder.py    
  inflating: __MACOSX/qtacotron/._vocoder.py  
   creating: qtacotron/audio-samples/
  inflating: __MACOSX/qtacotron/._audio-samples  
  inflating: qtacotron/LICENSE       
  inflating: __MACOSX/qtacotron/._LICENSE  
  inflating: qtacotron/demo.wav      
  inflating: __MACOSX/qtacotron/._demo.wav  
  inflating: qtacotron/hparams.py    
  inflating: __MACOSX/qtacotron/._hparams.py  
  inflating: qtacotron/preprocess.py  
  inflating: __MACOSX/qtacotron/._preprocess.py  
   creating: qtacotron/hifigan/
  inflating: __MACOSX/qtacotron/._hifigan  
   creating: qtacotron/process_utils/
  inflating: __MACOSX/qtacotron/._process_utils  
   creating: qtacotron/hifi-gan/
  inflating: __MACOSX/qtacotron/._hifi-gan  
  inflating: qtacotron/.gitmodules   
  inflating: __MACOSX/qtacotron/._.gitmodules  
  inflating: qtacotron/model.py      
  inflating: __MACOSX/qtaco

In [3]:
cd qtacotron

/content/qtacotron


# install pips

In [4]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 29.9 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install pyreaper

     |████████████████████████████████| 124 kB 5.2 MB/s 
  Created wheel for pyreaper: filename=pyreaper-0.0.8-cp37-cp37m-linux_x86_64.whl size=471652 sha256=dd89c244ef5027691c18f49fec9f20eaab991f22fd81111101f9e3bd2a62d6f8
  Stored in directory: /root/.cache/pip/wheels/18/c7/fe/71a7e52fe0110d625f57386b9597977fb058324eea5f6f784e
Successfully built pyreaper


#Downloads hifigan https://github.com/jik876/hifi-gan

In [6]:
!git clone https://github.com/jik876/hifi-gan

Cloning into 'hifi-gan'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Unpacking objects: 100% (48/48), done.


In [7]:
!gdown --id 1QEBKespXTmsMzsSRBXWdpIT0Ve7nnaRZ -O hifi-gan/

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1QEBKespXTmsMzsSRBXWdpIT0Ve7nnaRZ
To: /content/qtacotron/hifi-gan/generator_v1
100% 55.8M/55.8M [00:00<00:00, 205MB/s]


In [8]:
!gdown --id 1l5EUVBKM0SK7ec4HWf_wZvEITAsdOLFC -O hifi-gan/

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1l5EUVBKM0SK7ec4HWf_wZvEITAsdOLFC
To: /content/qtacotron/hifi-gan/config.json
100% 795/795 [00:00<00:00, 1.21MB/s]


In [9]:
rm -rf hifigan

In [10]:
!mv -f hifi-gan/ hifigan

# inference

In [11]:
from inference import load_hifigan,load_tacotron,split,predict_f0_clusters,compute_bert_embeddings,plot_data
from process_utils.make_features import prepare_bert
from hparams import Hparams
from model import Tacotron2
from IPython.display import Audio
import os
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.io.wavfile import read, write
import numpy as np

In [12]:
def sync(text_encoder_out,f0_labels,embeddings,device,generator,model):
    batch = [torch.tensor([text_encoder_out]).float().to(device), torch.tensor([f0_labels]).long().to(device),
            torch.tensor(embeddings).float().to(device)]

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(batch)
    plot_data((mel_outputs.float().data.cpu().numpy()[0],
                mel_outputs_postnet.float().data.cpu().numpy()[0],
                alignments.float().data.cpu().numpy()[0].T))
    with torch.no_grad():
        y = generator(mel_outputs_postnet).squeeze().cpu().numpy()

    write(os.path.join("gen", 'out.wav'), 24000, (y * 32768).astype('int16'))
    

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hparams = Hparams()
letter2index = {hparams.symbols[i]: i for i in range(len(hparams.symbols))}
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
embedder = AutoModel.from_pretrained('cointegrated/rubert-tiny')
embedder.to(device)
embedder.eval()
model = load_tacotron()
model.to(device)
generator = load_hifigan()
os.makedirs('gen', exist_ok=True)
text = "The buses arent the problem, they actually provide a solution."
text = "=" + text.lower() + "~"
tokens, embeddings = compute_bert_embeddings(text,tokenizer,embedder,device)

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Removing weight norm...


## Predict f0

In [16]:
print(text)
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

=the buses arent the problem, they actually provide a solution.~
5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## lower value

In [17]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,0] = 0
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## high volume

In [18]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,0] = 4
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## lower f0

In [19]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,1] = 0
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## high f0

In [20]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,1] = 4
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## faster

In [21]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,4] = 0
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## slow

In [22]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[f0_labels[:,0]!=5,4] = 4
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


## The buses aren't the <b>PROBLEM</b>, they actually provide a <b>SOLUTION</b>.

In [23]:
text_encoder_out, f0_labels = predict_f0_clusters(model, text, tokens, embeddings, [], [5,5,5,5,5],device)
f0_labels[54:62,:] = 4
f0_labels[21:28,:] = 4
sync(text_encoder_out,f0_labels,embeddings,device,generator,model)
Audio("gen/out.wav")

5333544444533333500052222222554444533333333522222225151111111155
5333533333533333522251111111552222533333333511111115150000000055
5222533333533333522251111111552222533333333522222225153333333355
5111511111511112533353333333552222511111111533333335351111111155
5000544444533332500053333333550000522222222522222225153333333355


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
